# Webscraping

> Below you will find examples of webscraping using 3 libraries.

⚠️ This writing is a work in progress.⚠️

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/bnia/dataguide/main?filepath=%2Fnotebooks%2F05_Webscraping.ipynb)
[![Binder](https://pete88b.github.io/fastpages/assets/badges/colab.svg)](https://colab.research.google.com/github/bnia/dataguide/blob/main/notebooks/05_Webscraping.ipynb)
[![Binder](https://pete88b.github.io/fastpages/assets/badges/github.svg)](https://github.com/bnia/dataguide/tree/main/notebooks/05_Webscraping.ipynb)
[![Open Source Love svg3](https://badges.frapsoft.com/os/v3/open-source.svg?v=103)](https://github.com/ellerbrock/open-source-badges/)

[![NPM License](https://img.shields.io/npm/l/all-contributors.svg?style=flat)](https://github.com/bnia/dataguide/blob/main/LICENSE)
[![Active](http://img.shields.io/badge/Status-Active-green.svg)](https://bnia.github.io) 
[![GitHub last commit](https://img.shields.io/github/last-commit/bnia/dataguide.svg?style=flat)]()  

[![GitHub stars](https://img.shields.io/github/stars/bnia/dataguide.svg?style=social&label=Star)](https://github.com/bnia/dataguide) 
[![GitHub watchers](https://img.shields.io/github/watchers/bnia/dataguide.svg?style=social&label=Watch)](https://github.com/bnia/dataguide) 
[![GitHub forks](https://img.shields.io/github/forks/bnia/dataguide.svg?style=social&label=Fork)](https://github.com/bnia/dataguide) 
[![GitHub followers](https://img.shields.io/github/followers/bnia.svg?style=social&label=Follow)](https://github.com/bnia/dataguide) 

[![Tweet](https://img.shields.io/twitter/url/https/github.com/bnia/dataguide.svg?style=social)](https://twitter.com/intent/tweet?text=Check%20out%20this%20%E2%9C%A8%20colab%20by%20@bniajfi%20https://github.com/bnia/dataguide%20%F0%9F%A4%97) 
[![Twitter Follow](https://img.shields.io/twitter/follow/bniajfi.svg?style=social)](https://twitter.com/bniajfi)

> note: There is a lot of quoting and paraphrasing on this page in particular. Be sure to check the links you want to learn more.

Selenium vs Puppetteer vs jsdom vs casper.ipynb

[casper](https://www.casperjs.org/) is missing. Very easy to use.


## [JSdom](https://github.com/jsdom/jsdom)

In [ ]:
!npm install jsdom

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ jsdom@16.6.0
added 61 packages from 95 contributors and audited 61 packages in 4.291s
found 0 vulnerabilities



In [ ]:
%%writefile testfile.js
const jsdom = require("jsdom");
const { JSDOM } = jsdom;
JSDOM.fromURL('https://bniajfi.org', {resources: "usable"}).then(dom => {
  txt = dom.serialize()
  txt=txt.split('document.getElementById("container")')[1]
  console.log( txt )
});


Writing testfile.js


In [ ]:
%%writefile bnia_trends.js
const jsdom = require("jsdom");
const { JSDOM } = jsdom;

(async () => { 
    
  links.map( link =>{
      var url = 'https://bniajfi.org'
      console.log(url)
      JSDOM.fromURL(url, {resources: "usable"}).then(dom => {
      txt = dom.serialize()
      txt=txt.split('document.getElementById("container")')[0] 
      console.log( txt )
    });
  } )
} )

Writing bnia_trends.js


## Puppetteer

Headless Recorder Extension

In [ ]:
%%capture
!npm install puppeteer

In [ ]:
%%capture
!apt install chromium-chromedriver

Now lets write the js script and stuff it into an index.js file

In [ ]:
# !npm install papaparse
! npm i -s csv-parser

In [ ]:
!npm i -s csv-writer

In [ ]:
! npm i async-csv

In [ ]:
!echo "" >data.csv

In [ ]:
! npm install html-table-to-json

In [ ]:
! pip install requests_html

In [ ]:
%%writefile bniatest.js
'use strict';

// Load our tool
const HtmlTableToJson = require('html-table-to-json');
const csv = require('csv-parser');
const fs = require('fs');
const puppeteer = require('puppeteer');

(async () => {
  // Setup the browser
  var browser = await puppeteer.launch({
    executablePath:"/usr/lib/chromium-browser/chromium-browser", 
    args:['--no-sandbox', `--window-size=1200,1000`],
    headless: false, // The browser is visible
    ignoreHTTPSErrors: true,
  });

  const page = await browser.newPage()
  async function getTopWallets(page, num){
    // Visit our page // Configure the page view.
    await page.goto(`https://bniajfi.org`)
    await page.setViewport({ width: 800, height: 70000 })
    await page.waitForTimeout(500);
    var tblz = await page.evaluate( () => {
        function getTblData(tblid){
          if(tblid=='#tblOne'){
            Array.prototype.forEach.call( document.querySelectorAll(tblid+ " > thead"), function( node ) {
                var el = document.createElement("thead");  el.innerHTML = header
                node.parentNode.insertBefore(el, node.nextSibling); node.parentNode.removeChild( node );
                return Array.from( document.querySelectorAll(tblid), element => element.outerHTML )
            });
          }
          Array.prototype.forEach.call( document.querySelectorAll(tblid +" > tbody > tr > td > span"), function( node ) { node.parentNode.removeChild( node ); });
          Array.prototype.forEach.call( document.querySelectorAll(tblid +" > tbody > tr > td > small"), function( node ) { node.parentNode.removeChild( node ); });
          Array.prototype.forEach.call( document.querySelectorAll(tblid +" > tbody > tr > td:nth-child(3)"), function( node ) { node.innerHTML = node.innerHTML; });
          var output =  Array.from( document.querySelectorAll(tblid), element => element.outerHTML )
          var position =  121
          if(tblid=='#tblOne2'){ output = [ [output[0].slice(0, position), header, output[0].slice(position)].join('') ] }
          return output
        }
        return [...getTblData('#tblOne'), ...getTblData('#tblOne2') ]
    } )
    return [...HtmlTableToJson.parse( tblz[0] ).results, ...HtmlTableToJson.parse( tblz[1] ).results ].flat()
  }
  var data = []
  for (var i = 1; i < 2; i++) {
    data = [...data, ...await getTopWallets(page, i==1?'':('-'+i) ) ]
  } 

  csvWriter.writeRecords(data).then(()=> console.log('The CSV file was written'));

  await browser.close()
})()

In [ ]:
!node bniatest.js

## [Selenium](https://stackoverflow.com/questions/55085072/how-to-use-selenium-on-colaboratory-google)

In [ ]:
!sudo apt install unzip
!wget https://chromedriver.storage.googleapis.com/2.37/chromedriver_linux64.zip
#!unzip chromedriver_linux64.zip -d /usr/bin/
!pip install selenium
!pip install openpyxl
!apt-get update
!apt-get install -y unzip xvfb libxi6 libgconf-2-4
!apt-get install default-jdk 

In [ ]:
! apt install chromium-chromedriver

In [ ]:
from google.colab import drive

In [ ]:
import os, subprocess
import sys
import selenium
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import openpyxl
import time, re, csv, urllib.parse
import pandas as pd

In [ ]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 200)
mobile_emulation = { "deviceName": "iPhone X" }

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
driver = webdriver.Chrome('chromedriver', chrome_options=options)

In [ ]:
driver.get('https://bniajfi.org')
time.sleep(.5)
driver.save_screenshot('screenie.png')

with open("./page_source.html", "w") as f:
    f.write(driver.page_source)

In [ ]:
driver.find_elements_by_css_selector('.container')

In [ ]:
driver.find_element_by_tag_name('html').get_attribute('innerHTML')

In [ ]:
time.sleep(.5)
login = driver.find_element_by_xpath('//*[@id="container"]/div/div/')
login.click()

#username
username = driver.find_element_by_xpath('//*[@id="container"]/div/div/')
username.click()
username.send_keys('user')

#password
password = driver.find_element_by_xpath('//*[@id="container"]/div/div/')
password.click()
password.send_keys('pass')

#sign in 
signin = driver.find_element_by_xpath('//*[@id="container"]/div/div/')
signin.click()

In [ ]:
import re
import os
import time
import random
import requests
import numpy as np
import pandas as pd
from os import system
from math import floor
from copy import deepcopy
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from selenium.webdriver.chrome.options import Options

import matplotlib.pyplot as plt

In [ ]:
# import scraping modules
from selenium import webdriver
import pandas as pd

# open browser
driver = webdriver.Chrome()

# scrape vital signs
driver.get("https://medicalprogress.dev/patient_file2/vit_signs.html")
html = driver.page_source
driver.close()
data = pd.read_html(html)
data = data[0]
data = pd.DataFrame(data)
print(data)

In [ ]:
# replace all comments behind numbers with nothing
data[data.columns[1]] = data[data.columns[1]].str.replace(r'[a-zA-Z].*', '', regex=True)